In [ ]:
import pandas as pd
import pickle
import os

import plotly.express as px
import plotly.graph_objects as go
import torch
import numpy as np

In [ ]:
mount_point = "/mnt/raid/C1_ML_Analysis/"



In [ ]:

df_test = pd.read_csv('/mnt/raid/C1_ML_Analysis/CSV_files/efw_2025-11-12_test.csv')


In [ ]:
df_test.columns

In [ ]:
prediction_sweep = pickle.load(open(os.path.join(mount_point, 'test_output/fetal_biometry/efw/v3.7/efw_2025-11-12_test/prediction_sweep.pkl'), 'rb'))
max_scores_sweep = [torch.max(s).item() for s in prediction_sweep['scores_sweeps']]
efw_pred_max = []
for idx in range(len(prediction_sweep['scores_sweeps'])):
    max_idx = torch.argmax(prediction_sweep['scores_sweeps'][idx])
    efw_pred_max.append(prediction_sweep['efw_pred_frames'][idx].view(-1)[max_idx].item())


In [ ]:
prediction_sweep.keys()

In [ ]:
df_prediction_sweep = pd.DataFrame({'file_path': prediction_sweep['file_path'], 'study_id': prediction_sweep['id'], 'efw_pred': prediction_sweep['efw_pred']*1000, 'efw_gt': prediction_sweep['efw_gt']*1000, 'max_scores_frames': max_scores_sweep, 'efw_pred_max': np.array(efw_pred_max)*1000})

df_prediction_sweep['efw_error'] = df_prediction_sweep['efw_pred'] - df_prediction_sweep['efw_gt']
df_prediction_sweep['abs_efw_error'] = df_prediction_sweep['efw_error'].abs()
df_prediction_sweep['efw_bin'] = pd.cut(df_prediction_sweep['efw_gt'], bins=[0,2000,3000,5000], labels=['0-2000','2000-3000','3000-5000'])

df_prediction_sweep = df_prediction_sweep.merge(df_test[['study_id', 'dataset']], on='study_id')

In [ ]:
px.violin(df_prediction_sweep[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution').show()

In [ ]:
px.violin(df_prediction_sweep.query('max_scores_frames > 0.5')[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution').show()

In [ ]:
px.violin(df_prediction_sweep.query('max_scores_frames > 0.8')[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution').show()

In [ ]:
df_prediction_sweep['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_prediction_sweep.query('max_scores_frames > 0.8')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_prediction_sweep.query('max_scores_frames > 0.8').groupby('efw_bin')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_max_score = df_prediction_sweep.loc[df_prediction_sweep.groupby('study_id')['max_scores_frames'].idxmax()]


In [ ]:
px.violin(df_max_score[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution').show()

In [ ]:
px.violin(df_max_score.query('max_scores_frames > 0.8')[['study_id', 'abs_efw_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution').show()

In [ ]:
df_max_score['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_max_score.query('max_scores_frames > 0.5')['abs_efw_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_prediction_sweep['efw_max_error'] = df_prediction_sweep['efw_pred_max'] - df_prediction_sweep['efw_gt']
df_prediction_sweep['abs_efw_max_error'] = df_prediction_sweep['efw_max_error'].abs()

In [ ]:
px.violin(df_prediction_sweep[['study_id', 'abs_efw_max_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_max_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution MAX FRAME ONLY').show()

In [ ]:
px.violin(df_prediction_sweep.query('max_scores_frames > 0.5')[['study_id', 'abs_efw_max_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_max_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution MAX FRAME ONLY').show()

In [ ]:
px.violin(df_prediction_sweep.query('max_scores_frames > 0.8')[['study_id', 'abs_efw_max_error', 'efw_bin', 'dataset']].drop_duplicates(), y='abs_efw_max_error', x='efw_bin', color='dataset', box=True, points='all', title='EFW Absolute Error Distribution MAX FRAME ONLY').show()

In [ ]:
list(df_max_score.query('max_scores_frames > 0.8 and abs_efw_error > 500')['file_path'])

In [ ]:
df_prediction_sweep.groupby('efw_bin')['abs_efw_max_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_prediction_sweep.query('max_scores_frames > 0.5').groupby('efw_bin')['abs_efw_max_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

In [ ]:
df_prediction_sweep.query('max_scores_frames > 0.8').groupby('efw_bin')['abs_efw_max_error'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])